The goal of this notebook is to take the ad hoc implementation in `MIPExample1.ipynb` and vectorize the operations such that any size Tyche problem can be converted to a MIP and solved.

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath("../src"))

In [2]:
import numpy             as np
import matplotlib.pyplot as pl
import pandas            as pd
import seaborn           as sb
import tyche             as ty

from copy            import deepcopy
from IPython.display import Image 

In [3]:
import cProfile
import timeit

In [82]:
from mip import Model, MINIMIZE, MAXIMIZE, BINARY, xsum
from itertools import product

In [5]:
designs = ty.Designs("data")
investments = ty.Investments("data")
designs.compile()
tranche_results = investments.evaluate_tranches(designs, sample_count=250)
results = investments.tranches.join(tranche_results.summary)
evaluator = ty.Evaluator(investments.tranches, tranche_results.summary)

Get the wide-format interpolated elicitation data from the Tyche Evaluator and reset the multi-level index.

In [6]:
wide = evaluator.evaluate_corners_wide().reset_index()

In [7]:
wide.head(9)

Index,CIGS,CdTe,GaAs,InGaP,Perovskite,Polysilicon,Power Electronics,Soft Costs,Capital,Efficiency,GHG,Hazardous,LCOE,Lifetime,Strategic,Yield
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.547360,2.066883,-0.003592,0.973466,-0.143004,188.051752,0.063428,10006.828512
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,-1.465742,2.066316,-0.003592,0.974118,-0.140451,188.051661,0.063428,10006.832792
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5000000.0,-1.295581,2.066924,-0.003592,0.973480,-0.135127,188.051679,0.063428,10006.827268
3,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,0.0,-1.473330,2.066936,-0.003592,0.961711,-0.136984,188.051696,0.063428,10070.983363
4,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,1000000.0,-1.391713,2.066369,-0.003592,0.962363,-0.134430,188.051605,0.063428,10070.987644
5,0.0,0.0,0.0,0.0,0.0,0.0,1000000.0,5000000.0,-1.221552,2.066978,-0.003592,0.961725,-0.129107,188.051623,0.063428,10070.982119
6,0.0,0.0,0.0,0.0,0.0,0.0,5000000.0,0.0,-1.400477,2.066152,-0.003592,0.955191,-0.132598,188.051757,0.063428,10113.752068
7,0.0,0.0,0.0,0.0,0.0,0.0,5000000.0,1000000.0,-1.318860,2.065585,-0.003592,0.955843,-0.130044,188.051665,0.063428,10113.756348
8,0.0,0.0,0.0,0.0,0.0,0.0,5000000.0,5000000.0,-1.148699,2.066194,-0.003592,0.955204,-0.124720,188.051684,0.063428,10113.750823


**Input to MIP constructor needed**: List of investment categories

**Input to MIP constructor needed**: List of metrics to optimize and/or constrain

**Data check**: Confirm that all elements of both input lists match columns in the `evaluate_corners_wide()` data frame.

In [8]:
categories = ['Polysilicon', 'Power Electronics', 'Soft Costs']

metrics = ['Capital']

Pull out the investment values and metric values from the data set.

In [53]:
# Investment levels
inv_levels = wide.loc[:26,categories].values.tolist()

# Elicited metric values
m = wide.loc[:26,metrics].values.tolist()

# Number of endpoints
I = len(inv_levels)

Instantiate the MIP optimization problem.

In [134]:
example = Model(sense=MAXIMIZE)

Create binary (integer) variables $y_{in_i}$.

In [135]:
bin_vars = []

for i in range(I-1):
    _name = 'y' + '_' + str(i)
    bin_vars += [example.add_var(name=_name, var_type=BINARY)]
    del _name

Create continuous $\lambda$ variables with lower bound 0.0 and upper bound 1.0

In [136]:
lmbd_vars = []

for i in range(I):
    _name = 'lmbd' + '_' + str(i)
    lmbd_vars += [example.add_var(name=_name, lb=0.0, ub=1.0)]
    del _name

**Question**: Is there a standard way we're setting up the budget constraint(s)? e.g. Will there always be as many budget constraints as there are investment categories or always only one budget constraint?

Define upper bound(s) for budget constraint(s).

In [137]:
B = 3000000.0

Create total budget constraint as a function of the $\lambda$ variables and the elicited investment levels from the data set.

In [138]:
example += xsum(lmbd_vars[i] * inv_levels[i][j] 
                for i in range(I)
                for j in range(len(inv_levels[i]))) <= B, 'Total Budget'

Create three sets of constraints:
* Convexity constraints on $\lambda$ variables: the $\lambda$s must sum to 1.
* Constrain binary $y$ variables within each category such that the decision variables lie within exactly one interval (also a sum-to-1 constraint).
* Interval constraints on $y$ variables and $\lambda$ variables. There will be as many interval constraints per investment category as there are linear intervals.

In [139]:
example += sum(lmbd_vars) == 1, 'Lambda Sum'
example += sum(bin_vars) == 1, 'Binary Sum'

In [140]:
for i in range(I-1):
    example += bin_vars[i] <= lmbd_vars[i] + lmbd_vars[i+1], 'Interval Constraint' + str(i)

**In progress/Not implemented from this point onward**

Create objective function: Capital (metric) as a function of $\lambda$s and $y$s.

Create a data frame of $\lambda$ variables by mapping every element in `v` to its corresponding $\lambda$, using column name (investment category) and investment level.

In [141]:
example.objective = xsum(m[i][0] * lmbd_vars[i] for i in range(I))

Save model to a `lp` file for manual examination.

In [142]:
example.write('example.lp')

Optimize.

In [143]:
example.optimize()

<OptimizationStatus.OPTIMAL: 0>

Print optimal objective function value, with a negative applied to reverse the -1.0 in the objective function definition (minimizing a negative -> maximize a positive).

In [144]:
example.objective_value

-0.7975715496378262

Get the optimal $\lambda$ and $y$ values.

In [145]:
for v in example.vars:
    print('{} : {}'.format(v.name, v.x))

y_0 : 0.0
y_1 : 0.0
y_2 : 0.0
y_3 : 0.0
y_4 : 0.0
y_5 : 0.0
y_6 : 0.0
y_7 : 0.0
y_8 : 0.0
y_9 : 0.0
y_10 : 0.0
y_11 : 0.0
y_12 : 0.0
y_13 : 0.0
y_14 : 0.0
y_15 : 0.0
y_16 : 0.0
y_17 : 0.0
y_18 : 0.0
y_19 : 0.0
y_20 : 0.0
y_21 : 0.0
y_22 : 0.0
y_23 : 0.0
y_24 : 0.0
y_25 : 1.0
lmbd_0 : 0.0
lmbd_1 : 0.0
lmbd_2 : 0.0
lmbd_3 : 0.0
lmbd_4 : 0.0
lmbd_5 : 0.0
lmbd_6 : 0.0
lmbd_7 : 0.0
lmbd_8 : 0.0
lmbd_9 : 0.0
lmbd_10 : 0.0
lmbd_11 : 0.0
lmbd_12 : 0.0
lmbd_13 : 0.0
lmbd_14 : 0.0
lmbd_15 : 0.0
lmbd_16 : 0.0
lmbd_17 : 0.0
lmbd_18 : 0.0
lmbd_19 : 0.0
lmbd_20 : 0.0
lmbd_21 : 0.0
lmbd_22 : 0.0
lmbd_23 : 0.0
lmbd_24 : 0.0
lmbd_25 : 0.0
lmbd_26 : 1.0


Verify that budget constraint is met.

Calculate the optimal investment levels.